In [24]:
# Dependencies
import requests
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats as stats
import time
from sqlalchemy import create_engine
# API Key and Postgres password
from config import apikey
from config import sql_pword

In [25]:
# Read in CSV file that contains high-level stock information

csv_file = "constituents-financials_csv.csv"
financial_df = pd.read_csv(csv_file)
financial_df.head()

,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,Earnings/Share,52 Week Low,52 Week High,Market Cap,EBITDA,Price/Sales,Price/Book,SEC Filings
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,7.92,259.77,175.490,138721055226,9.048000e+09,4.390271,11.34,http://www.sec.gov/cgi-bin/browse-edgar?action...
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,1.70,68.39,48.925,10783419933,6.010000e+08,3.575483,6.35,http://www.sec.gov/cgi-bin/browse-edgar?action...
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,0.26,64.60,42.280,102121042306,5.744000e+09,3.740480,3.19,http://www.sec.gov/cgi-bin/browse-edgar?action...
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,3.29,125.86,60.050,181386347059,1.031000e+10,6.291571,26.14,http://www.sec.gov/cgi-bin/browse-edgar?action...
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,5.44,162.60,114.820,98765855553,5.643228e+09,2.604117,10.62,http://www.sec.gov/cgi-bin/browse-edgar?action...


In [26]:
# Reduce the number of columns

new_financial_df = financial_df[['Symbol', 'Name', 'Sector', 'Price', 'Price/Earnings', 'Dividend Yield',\
                                '52 Week Low', '52 Week High']].copy()
new_financial_df.head()

,Symbol,Name,Sector,Price,Price/Earnings,Dividend Yield,52 Week Low,52 Week High
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,259.77,175.490
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,68.39,48.925
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,64.60,42.280
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,125.86,60.050
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,162.60,114.820


In [27]:
# Transform the dataframe for import

new_financial_df["52 Week Diff"] = new_financial_df["52 Week Low"] - new_financial_df["52 Week High"]
import_df = new_financial_df.rename(columns={"Symbol": "symbol", "Name": "company_name", "Sector": "sector", "Price": "price",
                        "Price/Earnings": "price_per_earnings", "Dividend Yield": "dividend_yield",
                        "52 Week Low": "year_low", "52 Week High": "year_high", "52 Week Diff": "year_diff"})
import_df['id'] = np.arange(len(import_df))
import_df.head()

,symbol,company_name,sector,price,price_per_earnings,dividend_yield,year_low,year_high,year_diff,id
0,MMM,3M Company,Industrials,222.89,24.31,2.332862,259.77,175.490,84.280,0
1,AOS,A.O. Smith Corp,Industrials,60.24,27.76,1.147959,68.39,48.925,19.465,1
2,ABT,Abbott Laboratories,Health Care,56.27,22.51,1.908982,64.60,42.280,22.320,2
3,ABBV,AbbVie Inc.,Health Care,108.48,19.41,2.499560,125.86,60.050,65.810,3
4,ACN,Accenture plc,Information Technology,150.51,25.47,1.714470,162.60,114.820,47.780,4


In [28]:
# Read in the stock data set
stock_data = pd.read_csv('tickers.csv')
stock_data.head()

,Symbol
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN


In [29]:
# Generate the base URL
# https://www.alphavantage.co/query?function=TIME_SERIES_DAILY&symbol=MSFT&apikey=demo

base_url = "https://www.alphavantage.co/query?"
params = {
    "function":"TIME_SERIES_MONTHLY",
    "symbol":"",   #REMOVE AFTER VERIFYING
    "apikey":apikey
}

In [30]:
# Implement the months we will want to add to the columns

Start_Month = "2019-10-31"
End_Month = "2020-03-31"
x = Start_Month
hardcode_months = ["2019-10-31", "2019-11-29", "2019-12-31", "2020-01-31",
                   "2020-02-28", "2020-03-31"]
table_columns = ["date_2019_10", "date_2019_11", "date_2019_12", "date_2020_01", "date_2020_02", "date_2020_03",]
column_dict = {}
for x in range(0, len(hardcode_months)):
    column_dict[hardcode_months[x]] = table_columns[x]
column_dict["Symbol"] = "symbol"

In [31]:
# Prepare the dataframe

for month in hardcode_months:
    stock_data[month]= ""
stock_data.head()

,Symbol,2019-10-31,2019-11-29,2019-12-31,2020-01-31,2020-02-28,2020-03-31
0,MMM,,,,,,
1,AOS,,,,,,
2,ABT,,,,,,
3,ABBV,,,,,,
4,ACN,,,,,,


In [ ]:
# API-hitting code 

x = 0
for index, row in stock_data[0:].iterrows():
    x = x + 1
    params['symbol']=row["Symbol"]
    response = requests.get(base_url, params=params).json()
    
    for month in hardcode_months: 
        try:
            stock_data.loc[index,month]= response["Monthly Time Series"][month]["4. close"]
        except:
            pass
    if x == 5:
        x = 0
        time.sleep(61)
               
stock_data.head()

In [32]:
# What the dataframe looks like to demonstrate.
stock_data = pd.read_csv('stock_export.csv')
stock_data.head()

,Symbol,2019-10-31,2019-11-29,2019-12-31,2020-01-31,2020-02-28,2020-03-31
0,MMM,164.99,169.77,176.42,158.66,149.24,136.51
1,AOS,49.68,48.40,47.64,42.69,39.55,37.81
2,ABT,83.61,85.45,86.86,87.14,77.03,78.91
3,ABBV,79.55,87.73,88.54,81.02,85.71,76.19
4,ACN,185.42,201.16,210.57,205.21,180.59,163.26


In [33]:
# Drop rows with missing information

edit_df = stock_data.copy()
edit_df.replace('', np.nan, inplace=True)
edit_df.drop
#edit_df.dropna(how="any", inplace = True)
edit_df.head()

,Symbol,2019-10-31,2019-11-29,2019-12-31,2020-01-31,2020-02-28,2020-03-31
0,MMM,164.99,169.77,176.42,158.66,149.24,136.51
1,AOS,49.68,48.40,47.64,42.69,39.55,37.81
2,ABT,83.61,85.45,86.86,87.14,77.03,78.91
3,ABBV,79.55,87.73,88.54,81.02,85.71,76.19
4,ACN,185.42,201.16,210.57,205.21,180.59,163.26


In [34]:
edit_df.dropna(how="any", inplace = True)
edit_df.head()

,Symbol,2019-10-31,2019-11-29,2019-12-31,2020-01-31,2020-02-28,2020-03-31
0,MMM,164.99,169.77,176.42,158.66,149.24,136.51
1,AOS,49.68,48.40,47.64,42.69,39.55,37.81
2,ABT,83.61,85.45,86.86,87.14,77.03,78.91
3,ABBV,79.55,87.73,88.54,81.02,85.71,76.19
4,ACN,185.42,201.16,210.57,205.21,180.59,163.26


In [36]:
# Add in a difference column

edit_df["diff"] = edit_df[hardcode_months[-1]]-edit_df[hardcode_months[0]]
edit_df.head()

,Symbol,2019-10-31,2019-11-29,2019-12-31,2020-01-31,2020-02-28,2020-03-31,diff
0,MMM,164.99,169.77,176.42,158.66,149.24,136.51,-28.48
1,AOS,49.68,48.40,47.64,42.69,39.55,37.81,-11.87
2,ABT,83.61,85.45,86.86,87.14,77.03,78.91,-4.70
3,ABBV,79.55,87.73,88.54,81.02,85.71,76.19,-3.36
4,ACN,185.42,201.16,210.57,205.21,180.59,163.26,-22.16
...,...,...,...,...,...,...,...,...
500,XYL,76.69,77.51,78.79,81.66,77.34,65.13,-11.56
501,YUM,101.71,100.67,100.73,105.77,89.25,68.53,-33.18
502,ZBH,138.23,145.28,149.68,147.90,136.15,101.08,-37.15
503,ZION,48.47,49.78,51.92,45.49,39.95,26.76,-21.71


In [37]:
# Add a primary key ID column

stock_import_df = edit_df.rename(columns=column_dict)
stock_import_df['id'] = np.arange(len(stock_import_df))
stock_import_df.head()

,symbol,date_2019_10,date_2019_11,date_2019_12,date_2020_01,date_2020_02,date_2020_03,diff,id
0,MMM,164.99,169.77,176.42,158.66,149.24,136.51,-28.48,0
1,AOS,49.68,48.40,47.64,42.69,39.55,37.81,-11.87,1
2,ABT,83.61,85.45,86.86,87.14,77.03,78.91,-4.70,2
3,ABBV,79.55,87.73,88.54,81.02,85.71,76.19,-3.36,3
4,ACN,185.42,201.16,210.57,205.21,180.59,163.26,-22.16,4


In [15]:
# Set up Postgres stuff

rds_connection_string = f"postgres:{sql_pword}@localhost:5432/finance_db"
engine = create_engine(f'postgresql://{rds_connection_string}')

In [38]:
# Set up the connection

conn = engine.connect()

In [17]:
# Verify table existence

engine.table_names()

['company_info', 'closing_prices']

In [39]:
# Import CSV information

import_df.to_sql(name='company_info', con=engine, if_exists='append', index=False)

In [40]:
# Import API JSON information

stock_import_df.to_sql(name='closing_prices', con=engine, if_exists='append', index=False)

In [41]:
# Query verification with most volatile

most_volatile = conn.execute('SELECT * from company_info ORDER BY "year_diff" DESC LIMIT 1')

In [42]:
for rowproxy in most_volatile:
    for column, value in rowproxy.items():
        print(column, value)

id 30
symbol AMZN
company_name Amazon.com Inc
sector Consumer Discretionary
price 1351
price_per_earnings 296
dividend_yield 0
year_low 1498
year_high 813
year_diff 686


In [23]:
# Close connection

conn.close()